In [17]:
from heapq import *

In [18]:
def load_maze(fname='maze.txt'):
    mazet = [l.strip() for l in open(fname)]
    maze = {}
    for y, row in enumerate(mazet):
        for x, cell in enumerate(row):
            maze[x, y] = cell
    return maze

In [31]:
def maze_neighbours(field, here, mow_cost=None):
    x0, y0 = here
    neighbours = [((x, y), 1)
            for x in range(x0-1, x0+2)
            for y in range(y0-1, y0+2)
            if (x, y) in field
            if x == x0 and y != y0 or x != x0 and y == y0
            if field[x, y] == '.'
           ]
    if mow_cost is not None:
        neighbours += [((x, y), mow_cost)
            for x in range(x0-1, x0+2)
            for y in range(y0-1, y0+2)
            if (x, y) in field
            if x == x0 and y != y0 or x != x0 and y == y0
            if field[x, y] == '#'
           ]
    return neighbours

In [20]:
def path_end(path):
    return path[-1]
    
def extend_path(path, item):
    return path + [item]

In [21]:
def show_path(maze, path, sparse=True):
    w = max(x for x, _ in maze.keys())
    h = max(y for _, y in maze.keys())
    for y in range(h+1):
        s = ''
        for x in range(w+1):
            if (x, y) in path:
                if maze[x, y] == '#':
                    s += '*'
                else:
                    s+= 'O'
            else:
                if sparse:
                    if maze[x, y] == '.':
                        s += ' '
                    else:
                        s += '.'# field[y][x]
                else:
                    s += maze[x, y]
        print(s)

In [22]:
def est_dist(here, there):
    return abs(here[0] - there[0]) + abs(here[1] - there[1])

In [32]:
def bfs(maze, start, goal):
    agenda = [extend_path([], start)]
    while agenda and path_end(agenda[0]) != goal:
        current = agenda[0]
        here = path_end(current)
        neighbours = maze_neighbours(maze, here)
#         print(here, neighbours, len(current), len(agenda))
        new_paths = [extend_path(current, n) for n, _ in neighbours if n not in current]
        agenda = agenda[1:] + new_paths
    if agenda:
        return agenda[0]
    else:
        return "No route found"

In [33]:
def astar(maze, start, goal, mow_cost=None):
    agenda = []
    closed = set()
    heappush(agenda, (est_dist(start, goal), 0, extend_path([], start)))
    while agenda and path_end(agenda[0][2]) != goal:
        _, cost, current = heappop(agenda)
        here = path_end(current)
        if here not in closed:
#             print(here, len(current), len(agenda), len(closed))
            closed.add(here)
            neighbours = maze_neighbours(maze, here, mow_cost=mow_cost)
            new_paths = [(cost + c + est_dist(n, goal), cost + c,
                          extend_path(current, n)) 
                         for n, c in neighbours 
                         if n not in closed]
            for np in new_paths:
                heappush(agenda, np)
    if agenda:
        return agenda[0]
    else:
        return "No route found"
    

In [34]:
def bestfs(maze, start, goal, mow_cost=None):
    agenda = []
    closed = set()
    heappush(agenda, (0, extend_path([], start)))
    while agenda and path_end(agenda[0][1]) != goal:
        cost, current = heappop(agenda)
        here = path_end(current)
        if here not in closed:
#             print(here, len(current), len(agenda), len(closed))
            closed.add(here)
            neighbours = maze_neighbours(maze, here, mow_cost=mow_cost)
            new_paths = [(cost + c,
                          extend_path(current, n)) 
                         for n, c in neighbours 
                         if n not in closed]
            for np in new_paths:
                heappush(agenda, np)
    if agenda:
        return agenda[0]
    else:
        return "No route found"
    

In [35]:
maze = load_maze('../../data/08-maze.txt')
START = (0, 0)
GOAL = (max(x for x, _ in maze.keys()), max(y for _, y in maze.keys()))

In [47]:
fp = bfs(maze, START, GOAL)
show_path(maze, fp, sparse=True)
len(fp) - 1 

OOOO.. . .   . .. ..  . .  .     .  .    .  .     
. .OO. . ...    .  ..   . .. . ....   .....   . ..
....O. . .   . ...    ...    ...  .. .. ..  ..... 
 .  OOOOO. ..... . .... . .... ..       .  ... .  
   .. ..OOO.       .  . . . .   .... ..... ..    .
 .... ....O  . ..... .. . . . ...       .     . ..
..  . .   O..... . .  .   .   . .... .... ....... 
 .. . ....O . .  .   .. ... ... ..   . .. .. . .. 
 .. . . . O.. . .. .... ..   ..  ... . .   .    . 
    ... ..OO. .  . . .   . ...  .. . . .. .. .... 
...  .. ...O. . ..   . . .   ..  . .            . 
    ..  . .O. .  . . . ... . .  .. . ... . ... .. 
... ...    OOO. .. ... . . ....      . ..... ...  
. . . .. ....O   .  .      . ..... ...  .  . .   .
  . .  ...   O.... .. ......  .. . . . .. .. ...  
. . . .. ....O.  .    . .    ..    . .     . . . .
    .  . .. .O.. . .... . ....... ..   .. .. . .  
...   ..  .  OO  .   .. .  .  .    ...  .... .   .
    .  .. ....O... ...    .. .... ...  ..      ...
......  . . ..O. . . . ..... ..

142

In [37]:
h, c, ap = astar(maze, START, GOAL, mow_cost=3)
show_path(maze, ap, sparse=True)
h, c, len(ap)

OOOO.. . .   . .. ..  . .  .     .  .    .  .     
. .OO. . ...    .  ..   . .. . ....   .....   . ..
....O. . .   . ...    ...    ...  .. .. ..  ..... 
 .  OOOOO. ..... . .... . .... ..       .  ... .  
   .. ..OOO.       .  . . . .   .... ..... ..    .
 .... ....O  . ..... .. . . . ...       .     . ..
..  . .   O..... . .  .   .   . .... .... ....... 
 .. . ....O . .  .   .. ... ... ..   . .. .. . .. 
 .. . . . O.. . .. .... ..   ..  ... . .   .    . 
    ... ..OO. .  . . .   . ...  .. . . .. .. .... 
...  .. ...O. . ..   . . .   ..  . .            . 
    ..  . .O. .  . . . ... . .  .. . ... . ... .. 
... ...    OOO. .. ... . . ....      . ..... ...  
. . . .. ....O   .  .      . ..... ...  .  . .   .
  . .  ...   O.... .. ......  .. . . . .. .. ...  
. . . .. ....O.  .    . .    ..    . .     . . . .
    .  . .. .O.. . .... . ....... ..   .. .. . .  
...   ..  .  OO  .   .. .  .  .    ...  .... .   .
    .  .. ....O... ...    .. .... ...  ..      ...
......  . . ..O. . . . ..... ..

(110, 110, 99)

In [38]:
h, c, ap = astar(maze, START, GOAL, mow_cost=1)
show_path(maze, ap, sparse=True)
h, c, len(ap)

O   .. . .   . .. ..  . .  .     .  .    .  .     
* .  . . ...    .  ..   . .. . ....   .....   . ..
*... . . .   . ...    ...    ...  .. .. ..  ..... 
O.       . ..... . .... . .... ..       .  ... .  
O  .. ..   .       .  . . . .   .... ..... ..    .
O.... ....   . ..... .. . . . ...       .     . ..
*.  . .    ..... . .  .   .   . .... .... ....... 
O.. . ....  . .  .   .. ... ... ..   . .. .. . .. 
O.. . . .  .. . .. .... ..   ..  ... . .   .    . 
O   ... ..  . .  . . .   . ...  .. . . .. .. .... 
*..  .. ... . . ..   . . .   ..  . .            . 
O   ..  . . . .  . . . ... . .  .. . ... . ... .. 
*.. ...       . .. ... . . ....      . ..... ...  
* . . .. ....    .  .      . ..... ...  .  . .   .
O . .  ...    .... .. ......  .. . . . .. .. ...  
* . . .. .... .  .    . .    ..    . .     . . . .
O   .  . .. . .. . .... . ....... ..   .. .. . .  
*..   ..  .      .   .. .  .  .    ...  .... .   .
O   .  .. .... ... ...    .. .... ...  ..      ...
*.....  . . .. . . . . ..... ..

(98, 98, 99)

In [48]:
h, c, ap = astar(maze, START, GOAL)
show_path(maze, ap)
h, c, len(ap)

OOOO.. . .   . .. ..  . .  .     .  .    .  .     
. .OO. . ...    .  ..   . .. . ....   .....   . ..
....O. . .   . ...    ...    ...  .. .. ..  ..... 
 .  OOOOO. ..... . .... . .... ..       .  ... .  
   .. ..OOO.       .  . . . .   .... ..... ..    .
 .... ....O  . ..... .. . . . ...       .     . ..
..  . .   O..... . .  .   .   . .... .... ....... 
 .. . ....O . .  .   .. ... ... ..   . .. .. . .. 
 .. . . . O.. . .. .... ..   ..  ... . .   .    . 
    ... ..OO. .  . . .   . ...  .. . . .. .. .... 
...  .. ...O. . ..   . . .   ..  . .            . 
    ..  . .O. .  . . . ... . .  .. . ... . ... .. 
... ...    OOO. .. ... . . ....      . ..... ...  
. . . .. ....O   .  .      . ..... ...  .  . .   .
  . .  ...   O.... .. ......  .. . . . .. .. ...  
. . . .. ....O.  .    . .    ..    . .     . . . .
    .  . .. .O.. . .... . ....... ..   .. .. . .  
...   ..  .  OO  .   .. .  .  .    ...  .... .   .
    .  .. ....O... ...    .. .... ...  ..      ...
......  . . ..O. . . . ..... ..

(142, 142, 143)

In [49]:
c, bp = bestfs(maze, START, GOAL, mow_cost=3)
show_path(maze, bp)
c, len(bp)

OOOO.. . .   . .. ..  . .  .     .  .    .  .     
. .OO. . ...    .  ..   . .. . ....   .....   . ..
....O. . .   . ...    ...    ...  .. .. ..  ..... 
 .  OOOOO. ..... . .... . .... ..       .  ... .  
   .. ..OOO.       .  . . . .   .... ..... ..    .
 .... ....O  . ..... .. . . . ...       .     . ..
..  . .   O..... . .  .   .   . .... .... ....... 
 .. . ....O . .  .   .. ... ... ..   . .. .. . .. 
 .. . . . O.. . .. .... ..   ..  ... . .   .    . 
    ... ..OO. .  . . .   . ...  .. . . .. .. .... 
...  .. ...O. . ..   . . .   ..  . .            . 
    ..  . .O. .  . . . ... . .  .. . ... . ... .. 
... ...    OOO. .. ... . . ....      . ..... ...  
. . . .. ....O   .  .      . ..... ...  .  . .   .
  . .  ...   O.... .. ......  .. . . . .. .. ...  
. . . .. ....O.  .    . .    ..    . .     . . . .
    .  . .. .O.. . .... . ....... ..   .. .. . .  
...   ..  .  OO  .   .. .  .  .    ...  .... .   .
    .  .. ....O... ...    .. .... ...  ..      ...
......  . . ..O. . . . ..... ..

(110, 99)

In [50]:
c, bp = bestfs(maze, START, GOAL)
show_path(maze, bp)
c, len(bp)

OOOO.. . .   . .. ..  . .  .     .  .    .  .     
. .OO. . ...    .  ..   . .. . ....   .....   . ..
....O. . .   . ...    ...    ...  .. .. ..  ..... 
 .  OOOOO. ..... . .... . .... ..       .  ... .  
   .. ..OOO.       .  . . . .   .... ..... ..    .
 .... ....O  . ..... .. . . . ...       .     . ..
..  . .   O..... . .  .   .   . .... .... ....... 
 .. . ....O . .  .   .. ... ... ..   . .. .. . .. 
 .. . . . O.. . .. .... ..   ..  ... . .   .    . 
    ... ..OO. .  . . .   . ...  .. . . .. .. .... 
...  .. ...O. . ..   . . .   ..  . .            . 
    ..  . .O. .  . . . ... . .  .. . ... . ... .. 
... ...    OOO. .. ... . . ....      . ..... ...  
. . . .. ....O   .  .      . ..... ...  .  . .   .
  . .  ...   O.... .. ......  .. . . . .. .. ...  
. . . .. ....O.  .    . .    ..    . .     . . . .
    .  . .. .O.. . .... . ....... ..   .. .. . .  
...   ..  .  OO  .   .. .  .  .    ...  .... .   .
    .  .. ....O... ...    .. .... ...  ..      ...
......  . . ..O. . . . ..... ..

(142, 143)

In [51]:
def path_cost(maze, path, mow_cost=1):
    total = -1
    for x, y in path:
        if maze[x, y] == '#':
            total += mow_cost
        else:
            total += 1
    return total

In [52]:
path_cost(maze, ap, 3), len(ap)

(142, 143)

In [53]:
path_cost(maze, bp), len(bp)

(142, 143)

In [59]:
%%timeit
bfs(maze, START, GOAL)

9.54 ms ± 72.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [64]:
%%timeit
astar(maze, START, GOAL, mow_cost=3)

26.2 ms ± 249 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [61]:
%%timeit
bestfs(maze, START, GOAL, mow_cost=3)

32.8 ms ± 494 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [62]:
%%timeit
astar(maze, START, GOAL)

5.01 ms ± 30.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [63]:
%%timeit
bestfs(maze, START, GOAL)

7.65 ms ± 56 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
